In [39]:
import pandas as pd

pedidoDF = pd.read_excel("userData\arieteFinal.xlsx", index_col=0)
pedidoDF["Total do Pedido"] = pd.to_numeric(
    pedidoDF["Total do Pedido"], errors="coerce"
)
pedidoDF["Valor Pago"] = pd.to_numeric(pedidoDF["Valor Pago"], errors="coerce")
pedidoDF = (
    pedidoDF.groupby("Nome")
    .agg(
        {"Total do Pedido": "sum", "Valor Pago": "sum", "Número para contato": "first"}
    )
    .reset_index()
)

In [40]:
def whatsAppContact(number):
    number = str(number)
    splited = [
        "".join(filter(str.isdigit, i))
        for i in number.replace("-", "").replace(")", " ").split(" ")
        if i
    ]
    if len(splited) == 2:
        if len(splited[0]) not in [2, 3]:
            out = False
        elif len(splited[1]) not in [8, 9]:
            out = False
        else:
            splited[0] = splited[0].zfill(3)
            splited[1] = splited[1].zfill(9)
            out = "+55" + "".join(splited)
    elif len(splited) == 3:
        if len(splited[0]) > 2:
            out = False
        elif len(splited[1]) not in [2, 3]:
            out = False
        elif len(splited[2]) not in [8, 9]:
            out = False
        else:
            splited[0] = splited[0].zfill(2)
            splited[1] = splited[1].zfill(3)
            splited[2] = splited[2].zfill(9)
            out = "".join(splited)
    elif len(splited) == 1:
        if len(splited[0]) in [10, 11, 12]:
            out = "+55" + splited[0]
        elif len(splited[0]) in [12, 13, 14]:
            out = splited[0]
    else:
        out = False
    if out:
        return out
    else:
        return False

In [41]:
filtroDF = pedidoDF.query("`Total do Pedido` > `Valor Pago`")
filtroDF["Restante"] = filtroDF["Total do Pedido"] - filtroDF["Valor Pago"]
filtroDF["Número para contato"] = filtroDF["Número para contato"].apply(whatsAppContact)
filtroDF

C:\Users\allan\AppData\Local\Temp\ipykernel_14176\1450090741.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtroDF["Restante"] = filtroDF["Total do Pedido"] - filtroDF["Valor Pago"]
C:\Users\allan\AppData\Local\Temp\ipykernel_14176\1450090741.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtroDF["Número para contato"] = filtroDF["Número para contato"].apply(whatsAppContact)


,Nome,Total do Pedido,Valor Pago,Número para contato,Restante
1,Arthur BACO,70,35.0,+55087096359574,35.0
2,Beatriz Galvão Costa Ribeiro,50,25.0,+5574999217070,25.0
3,Camila Stefani dos Santos Silva,90,45.0,+5587996775641,45.0
5,Davi Rodrigues dos Santos,50,25.0,+55074988652884,25.0
8,Ellen Bringel,75,50.0,+5587996365835,25.0
10,Erick Deivid,50,25.0,+55074981177518,25.0
11,Ezequiel de Souza Silva,90,45.0,+5574998039456,45.0
13,Iury,92,46.0,+5574991270197,46.0
15,Joao Gabriel Amorim Barbosa,85,42.5,+5574991393530,42.5
16,José Matheus,105,52.5,+55089994550159,52.5


In [48]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

driver = webdriver.Chrome(options=Options().add_argument("--incognito"))
driver.get("https://web.whatsapp.com/")
driver.implicitly_wait(0.5)
try:
    while 1:
        driver.find_element(value="landing-main", by=By.CLASS_NAME)
        driver.implicitly_wait(1)
except:
    print("Login OK")

Login OK


In [68]:
import urllib.parse
import time


for index, row in filtroDF.iterrows():
    nome = row["Nome"].split(" ")[0]
    numero = row["Número para contato"]
    valor = row["Restante"]
    msg = f"""Bom dia, {nome}.
A fábrica informou que as camisas da atlética ficarão prontas essa semana.
O pagamento da segunda parte seria até o dia 05/03, mas aproveitando queríamos ir buscar até dia 01/03.
Se for possível para você adiantar segunda parte do pedido (R${valor}), agradeço. 
Qualquer problema estou à disposição."""

    driver.get(
        f"https://web.whatsapp.com/send?phone={numero}"
        + f"&text={urllib.parse.quote_plus(msg)}"
    )
    time.sleep(10)
    driver.find_element(By.XPATH, '//button[@aria-label="Enviar"]').click()
    time.sleep(10)

Bom dia, Arthur.
Recebemos a informação que as camisas da atlética ficarão prontas antes do previsto e conseguiremos ir lá pegar essa sexta agora (01/03).
O pagamento da segunda parte seria dia 05/03, porém com o adiantamento da loja já podemos pagar e pegar as blusas.
Referente ao seu pedido, falta o pagamento de R$35.0
Qualquer problema estou à disposição.
Bom dia, Beatriz.
Recebemos a informação que as camisas da atlética ficarão prontas antes do previsto e conseguiremos ir lá pegar essa sexta agora (01/03).
O pagamento da segunda parte seria dia 05/03, porém com o adiantamento da loja já podemos pagar e pegar as blusas.
Referente ao seu pedido, falta o pagamento de R$25.0
Qualquer problema estou à disposição.
Bom dia, Camila.
Recebemos a informação que as camisas da atlética ficarão prontas antes do previsto e conseguiremos ir lá pegar essa sexta agora (01/03).
O pagamento da segunda parte seria dia 05/03, porém com o adiantamento da loja já podemos pagar e pegar as blusas.
Referen

In [50]:
driver.get(f"https://web.whatsapp.com/send?phone=+5531998648191")

In [61]:
driver.find_element(By.XPATH, '//button[@aria-label="Enviar"]').click()